In [1]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
import pyspark.pandas as ps

/home/pedro/miniconda3/envs/myenvgpu/lib/python3.11/site-packages/pyspark/pandas/__init__.py:43: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


# Load Data

In [ ]:
spark = SparkSession.builder \
    .appName("Read JSON") \
    .getOrCreate()

df_data = spark.read.json("../data/processed/processed_data.json").pandas_api()
df_data = df_data.sort_values(by=['account_id', 'offer_id'])
df_data.head(20)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/13 22:25:45 WARN Utils: Your hostname, pedro-pc, resolves to a loopback address: 127.0.1.1; using 192.168.15.69 instead (on interface wlo1)
26/01/13 22:25:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/13 22:25:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 22:25:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


,account_id,age,channels,credit_card_limit,discount_value,duration,event,gender,min_value,offer_id,offer_type,registered_on,reward,time_since_test_start
0,6e014185620b49bd98749f728747572f,75,"[web, email, mobile, social]",40000.0,5,5.0,offer received,M,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,20151112,NaN,0.0
1,70be3c98f6f0499ab6fd1c1e487094b5,67,"[email, mobile, social]",99000.0,0,3.0,offer received,O,0,5a8bc65990b245e5a138643cd4eb9837,informational,20170710,NaN,0.0
2,aef583f3c3614f9e96d7940df55510b2,20,"[web, email, mobile, social]",75000.0,5,5.0,offer received,M,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,20180427,NaN,0.0
3,09eb3e5e84224d018376b4d92b00b815,54,"[web, email, mobile]",35000.0,5,7.0,offer received,M,5,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,20180402,NaN,0.0
4,5ed60175a25c4fdf9c1b6ca43409e03b,55,"[web, email, mobile]",71000.0,0,4.0,offer received,F,0,3f207df678b143eea3cee63160fa8bed,informational,20180125,NaN,0.0
5,6a3176bdc38a41b39b7c186f6b2457a9,79,"[email, mobile, social]",72000.0,10,7.0,offer received,M,10,ae264e3637204a6fb9bb56bc8210ddfd,bogo,20160518,NaN,0.0
6,01956670cf414b309675aa73368b94a9,52,"[web, email, mobile, social]",95000.0,10,5.0,offer received,M,10,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,20160825,NaN,0.0
7,0c88140597254428ba857b1e6782bce1,27,"[web, email]",50000.0,5,10.0,offer received,M,20,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,20131010,NaN,0.0
8,66f5ffa82c1d49019aa884922d0c3c92,37,"[web, email, mobile]",31000.0,2,7.0,offer received,M,10,2906b810c7d4411798c6938adc9daaa5,discount,20160130,NaN,0.0
9,b11d014250274688834d40d8f94f66a0,60,"[web, email, mobile, social]",56000.0,2,10.0,offer received,M,10,fafdcd668e3743c1bb461111dcafc2a4,discount,20160204,NaN,0.0


# Approach 1: Classification of current transaction

In this approach, we will train a classifier to 